# Tutorial 1: Phase Shift Calculation for Ni(111) - Quick Start Guide

## Introduction

This tutorial demonstrates how to calculate electron scattering phase shifts for Low-Energy Electron Diffraction (LEED) using the `phaseshifts` package. We will use Nickel(111) as our example system, which is a common surface science benchmark.

### What are Phase Shifts?

Phase shifts describe how electrons scatter elastically from atoms in a crystal. In LEED experiments, an electron beam strikes a surface and diffracts. The resulting diffraction pattern depends on:

1. **The crystal structure** - atomic positions and lattice parameters
2. **The scattering properties of atoms** - encoded as phase shifts

Phase shifts are energy-dependent quantities $\delta_l(E)$ for each angular momentum channel $l = 0, 1, 2, ...$ (s, p, d, f, ...). They determine the scattering amplitude for each partial wave.

### The Barbieri/Van Hove Methodology

The `phaseshifts` package implements the [Barbieri/Van Hove phase shift calculation package](https://www.icts.hkbu.edu.hk/VanHove_files/leed/leedpack.html), which follows these steps:

1. **Step 0 (phsh0)**: Calculate atomic orbital charge densities using Dirac-Fock self-consistent field method
2. **Step 1 (phsh1)**: Calculate the muffin-tin potential by superposing atomic charge densities
3. **Step 2 (phsh2)**: Solve the radial Schrodinger/Dirac equation to obtain phase shifts
4. **Step 3 (phsh3/conphas)**: Remove $\pi$ discontinuities to produce continuous phase shifts

### References

- Barbieri, A. & Van Hove, M.A. (1979). "Phase shift calculation package for LEED." [Surf. Sci. 90, 1-25](https://doi.org/10.1016/0039-6028(79)90470-7)
- Van Hove, M.A. et al. (1986). "Surface Crystallography by LEED." Springer-Verlag
- Pendry, J.B. (1974). "Low Energy Electron Diffraction." Academic Press

## Setup

First, let's install and import the required packages.

In [ ]:
# Install phaseshifts if not already installed
%pip install -q phaseshifts numpy matplotlib

In [ ]:
import os
import tempfile
import numpy as np
import matplotlib.pyplot as plt

# Import phaseshifts modules
from phaseshifts.phsh import Wrapper
from phaseshifts.conphas import Conphas

# Set up plotting style
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12

print(f"phaseshifts version: {__import__('phaseshifts').__version__}")

## Understanding the Input Files

The `phaseshifts` package uses CLEED-style input files. Two files are required:

1. **Bulk file (`.bul`)**: Describes the 3D bulk unit cell
2. **Slab file (`.inp`)**: Describes the surface slab (2D + layers)

### Ni(111) Crystal Structure

Nickel has an FCC (face-centered cubic) structure with:
- Lattice constant: a = 3.52 Angstrom
- For the (111) surface, the surface unit cell is hexagonal
- The interlayer spacing is $d_{111} = a/\sqrt{3} \approx 2.033$ Angstrom

In [ ]:
# Let's visualize the Ni(111) structure (optional - requires ASE)
try:
    from ase.build import fcc111
    from ase.visualize.plot import plot_atoms
    
    # Create a Ni(111) slab with 3 layers
    ni_slab = fcc111('Ni', size=(3, 3, 4), a=3.52, vacuum=10.0)
    
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    
    # Top view
    plot_atoms(ni_slab, axes[0], rotation='0x,0y,0z')
    axes[0].set_title('Ni(111) - Top View')
    axes[0].set_xlabel('x (Angstrom)')
    axes[0].set_ylabel('y (Angstrom)')
    
    # Side view
    plot_atoms(ni_slab, axes[1], rotation='90x,0y,0z')
    axes[1].set_title('Ni(111) - Side View')
    axes[1].set_xlabel('x (Angstrom)')
    axes[1].set_ylabel('z (Angstrom)')
    
    plt.tight_layout()
    plt.show()
    
    print(f"Ni(111) slab created with {len(ni_slab)} atoms")
    print(f"Lattice constant: a = 3.52 Angstrom")
    print(f"Interlayer spacing: d_111 = {3.52/np.sqrt(3):.4f} Angstrom")
except ImportError:
    print("ASE not installed. Skipping structure visualization.")
    print("Install with: pip install ase")

### Bulk Input File Format

The bulk file defines the 3D periodicity of the crystal. Key parameters:

| Parameter | Description |
|-----------|-------------|
| `c:` | Comment line (model name) |
| `a1:`, `a2:`, `a3:` | Lattice vectors in Angstrom |
| `m1:`, `m2:` | Superstructure matrix (usually identity for clean surfaces) |
| `sr:` | Search parameters (symmetry) |
| `vr:`, `vi:` | Real and imaginary parts of the optical potential (eV) |
| `pb:` | Bulk atomic positions with element tag and coordinates |
| `ei:`, `ef:`, `es:` | Initial, final energy and step (eV) |
| `lm:` | Maximum angular momentum quantum number (l_max) |

In [ ]:
# Define the Ni(111) bulk input file content
bulk_content = '''# Ni(111) bulk geometry input file
c: Ni(111) - FCC Nickel (111) surface
#
# Lattice vectors for hexagonal surface unit cell
# a = 3.52 Angstrom (FCC lattice constant)
# Surface lattice: a_surf = a/sqrt(2) = 2.49 Angstrom
#
a1:       1.2450  -2.1564   0.0000
a2:       1.2450   2.1564   0.0000
a3:       0.0000   0.0000  -6.0990
#
# Superstructure matrix (1x1 for clean surface)
m1:  1. 0.
m2:  0. 1.
#
# Search symmetry
sr: 3  0.0  0.0
#
# Optical potential (eV)
# vr: real part (inner potential)
# vi: imaginary part (absorption)
vr:   -8.00
vi:     4.00
#
# Bulk atomic positions (ABC stacking for FCC)
# Format: pb: <element_tag> <x> <y> <z> dr3 <dx> <dy> <dz>
# The dr3 parameters define search ranges for refinement
pb: Ni_BVH  0.0000    +0.0000   0.0000  dr3 0.025 0.025 0.025
pb: Ni_BVH  1.2450    -0.7188  -2.0330  dr3 0.025 0.025 0.025
pb: Ni_BVH  1.2450    +0.7188  -4.0660  dr3 0.025 0.025 0.025
#
# Energy range for phase shift calculation
ei: 70.
ef: 498.1
es: 4.
#
# Other parameters
it: 0.
ip: 0.
ep: 1.e-2
#
# Maximum angular momentum (l_max)
# Rule of thumb: l_max ~ sqrt(E_max/1Ryd) ~ sqrt(500/13.6) ~ 6
# Use higher for accuracy (9-12 typical)
lm: 10
'''

print("=== Ni(111) Bulk File ===")
print(bulk_content)

### Slab Input File Format

The slab file defines the surface model. Key differences from bulk:

| Parameter | Description |
|-----------|-------------|
| `a1:`, `a2:` | 2D surface lattice vectors |
| `po:` | Overlayer/surface atomic positions |
| `rm:` | Muffin-tin radius for each element (Angstrom) |
| `zr:` | z-range for the slab (bottom, top) in Angstrom |
| `sz:` | Number of composite layers in z direction |

In [ ]:
# Define the Ni(111) slab input file content
slab_content = '''# Ni(111) slab geometry input file
#
# 2D surface lattice vectors
a1:       1.2450        2.1564    0.0000
a2:       1.2450       -2.1564    0.0000
#
# Superstructure matrix
m1:  1. 0.
m2:  0. 1.
#
# Surface/overlayer atomic positions
# Note: z-coordinates are positive (surface is at top)
po: Ni_BVH  0.0000    +0.0000   6.0990  dr3 0.025 0.025 0.025
po: Ni_BVH  1.2450    -0.7188   4.0660  dr3 0.025 0.025 0.025
po: Ni_BVH  1.2450    +0.7188   2.0330  dr3 0.025 0.025 0.025
#
# Muffin-tin radii (Angstrom)
# Typical values: 0.7-1.0 * nearest-neighbor distance / 2
# For Ni: d_NN = a/sqrt(2) = 2.49 A, so r_MT ~ 0.9-1.1 A
rm: Ni_BVH  0.90
#
# z-range for slab (defines vacuum region)
zr: 1.60  7.00
#
# Number of composite layers
sz: 1
#
# Symmetry
sr: 3 0.0 0.0
'''

print("=== Ni(111) Slab File ===")
print(slab_content)

## Running the Phase Shift Calculation

Now we'll run the complete phase shift calculation using `phsh.py`. This orchestrates all four steps automatically.

In [ ]:
# Create a temporary working directory
work_dir = tempfile.mkdtemp(prefix='phaseshifts_ni111_')
print(f"Working directory: {work_dir}")

# Write the input files
bulk_file = os.path.join(work_dir, 'Ni111.bul')
slab_file = os.path.join(work_dir, 'Ni111.inp')

with open(bulk_file, 'w') as f:
    f.write(bulk_content)
    
with open(slab_file, 'w') as f:
    f.write(slab_content)

print(f"Created bulk file: {bulk_file}")
print(f"Created slab file: {slab_file}")

In [ ]:
# Run the phase shift calculation
# This performs all four steps of the Barbieri/Van Hove workflow

print("Starting phase shift calculation...")
print("="*50)

try:
    output_files = Wrapper.autogen_from_input(
        bulk_file=bulk_file,
        slab_file=slab_file,
        tmp_dir=work_dir,
        model_name='Ni111',
        format='curve',  # Output format: 'cleed', 'curve', or 'viperleed'
        lmax=10,         # Maximum angular momentum
        range=(20.0, 500.0, 5.0),  # Energy range: start, stop, step (eV)
        verbose=True,
        store=work_dir,
    )
    
    print("="*50)
    print("Calculation completed!")
    print(f"Output files: {output_files}")
except Exception as e:
    print(f"Calculation failed: {e}")
    print("\nNote: This calculation requires the compiled Fortran library.")
    print("For demonstration, we'll use pre-computed example data.")
    output_files = []

## Understanding the Output

The calculation produces several intermediate and final output files:

| File | Description |
|------|-------------|
| `at_Ni.i` | Atomic charge density for Ni (Step 0) |
| `*_mufftin.d` | Muffin-tin potential (Step 1) |
| `*_phasout.i` | Raw phase shifts (Step 2) |
| `*.phs` or `*.cur` | Continuous phase shifts (Step 3) |

### Output Formats

The package supports multiple output formats:

1. **CLEED format (`.phs`)**: Used by the CLEED package
2. **Curve format (`.cur`)**: Simple energy-phase pairs for plotting
3. **ViPErLEED format**: Used by the Vienna LEED package

In [ ]:
# List generated files
import glob

print("Files in working directory:")
print("="*50)

for pattern in ['*.i', '*.d', '*.phs', '*.cur', '*.mtz', '*.bmtz']:
    files = glob.glob(os.path.join(work_dir, pattern))
    if files:
        print(f"\n{pattern} files:")
        for f in sorted(files):
            size = os.path.getsize(f)
            print(f"  {os.path.basename(f):40s} ({size:,} bytes)")

## Visualizing Phase Shift Curves

Phase shifts are typically plotted as a function of energy for each angular momentum channel. Let's visualize the results.

In [ ]:
def read_curve_file(filepath):
    """Read a .cur phase shift file and return energy and phase data."""
    try:
        data = np.loadtxt(filepath)
        if data.ndim == 1:
            # Single column - need to reshape
            return None, None
        energy = data[:, 0]  # First column is energy
        phases = data[:, 1:]  # Remaining columns are phase shifts for each l
        return energy, phases
    except Exception as e:
        print(f"Error reading {filepath}: {e}")
        return None, None

def read_cleed_phs_file(filepath):
    """Read a .phs CLEED format phase shift file."""
    try:
        with open(filepath, 'r') as f:
            lines = f.readlines()
        
        # First line contains neng and lmax
        header = lines[0].split()
        neng = int(header[0])
        lmax = int(header[1])
        
        # Parse the rest as phase shift data
        data = []
        for line in lines[1:]:
            values = [float(x) for x in line.split()]
            data.extend(values)
        
        # Reshape: first value in each row is energy, rest are phases
        data = np.array(data).reshape(neng, lmax + 2)  # energy + (lmax+1) phases
        energy = data[:, 0]
        phases = data[:, 1:]
        
        return energy, phases
    except Exception as e:
        print(f"Error reading {filepath}: {e}")
        return None, None

In [ ]:
# Try to load and plot the phase shift curves
cur_files = glob.glob(os.path.join(work_dir, '*.cur'))
phs_files = glob.glob(os.path.join(work_dir, '*.phs'))

# Angular momentum labels
l_labels = ['s (l=0)', 'p (l=1)', 'd (l=2)', 'f (l=3)', 'g (l=4)', 
            'h (l=5)', 'i (l=6)', 'k (l=7)', 'l (l=8)', 'm (l=9)', 'n (l=10)']

fig, ax = plt.subplots(figsize=(12, 8))

# Use a colormap for the different l channels
colors = plt.cm.tab10(np.linspace(0, 1, 11))

if cur_files:
    for cur_file in cur_files:
        energy, phases = read_curve_file(cur_file)
        if energy is not None:
            for l_idx in range(min(phases.shape[1], 8)):  # Plot up to l=7
                ax.plot(energy, phases[:, l_idx], 
                       color=colors[l_idx], 
                       label=l_labels[l_idx],
                       linewidth=2)
                       
elif phs_files:
    for phs_file in phs_files:
        energy, phases = read_cleed_phs_file(phs_file)
        if energy is not None:
            for l_idx in range(min(phases.shape[1], 8)):
                ax.plot(energy, phases[:, l_idx], 
                       color=colors[l_idx], 
                       label=l_labels[l_idx],
                       linewidth=2)
else:
    # Generate example phase shifts for demonstration
    print("No output files found. Generating example phase shift curves...")
    
    energy = np.linspace(20, 500, 97)
    k = np.sqrt(energy / 13.6)  # Wavevector in atomic units
    
    # Simplified model phase shifts (for illustration)
    for l in range(8):
        # Phase shifts typically decrease with l and have characteristic shapes
        delta_l = np.arctan(1.0 / (k + l + 1)) * (5 - l/2) + l * 0.5 * np.exp(-k/5)
        ax.plot(energy, delta_l, color=colors[l], label=l_labels[l], linewidth=2)

ax.set_xlabel('Energy (eV)', fontsize=14)
ax.set_ylabel('Phase Shift (radians)', fontsize=14)
ax.set_title('Ni(111) Phase Shifts vs Energy', fontsize=16)
ax.legend(loc='upper right', ncol=2, fontsize=10)
ax.set_xlim(20, 500)
ax.axhline(y=0, color='gray', linestyle='--', alpha=0.5)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(os.path.join(work_dir, 'ni111_phase_shifts.png'), dpi=150)
plt.show()

print(f"\nPlot saved to: {os.path.join(work_dir, 'ni111_phase_shifts.png')}")

## Key Parameters and Their Effects

### Energy Range

The energy range should cover the experimental LEED energy range:
- **Typical range**: 20-500 eV
- **Step size**: 2-5 eV (smaller for higher accuracy)

### Maximum Angular Momentum (l_max)

Higher l_max gives more accurate results but increases computation time:

| Energy Range | Recommended l_max |
|--------------|-------------------|
| < 100 eV     | 6-8               |
| 100-300 eV   | 8-10              |
| 300-500 eV   | 10-12             |

Rule of thumb: $l_{max} \approx \sqrt{E_{max}/R_y}$ where $R_y = 13.6$ eV

### Muffin-Tin Radius

The muffin-tin radius affects the potential and scattering:
- **Too small**: May miss important scattering contributions
- **Too large**: Spheres may overlap (unphysical)
- **Typical values**: 70-90% of half the nearest-neighbor distance

In [ ]:
# Calculate recommended muffin-tin radius for Ni
a_Ni = 3.52  # Angstrom (FCC lattice constant)
d_NN = a_Ni / np.sqrt(2)  # Nearest-neighbor distance

print("Muffin-tin radius considerations for Ni:")
print(f"  Lattice constant: a = {a_Ni:.3f} Angstrom")
print(f"  Nearest-neighbor distance: d_NN = {d_NN:.3f} Angstrom")
print(f"  Maximum MT radius (touching spheres): {d_NN/2:.3f} Angstrom")
print(f"  Recommended MT radius (70-90%): {0.7*d_NN/2:.3f} - {0.9*d_NN/2:.3f} Angstrom")
print(f"  Value used in tutorial: 0.90 Angstrom")

## Command-Line Interface

The `phsh.py` script can also be run from the command line:

In [ ]:
# Show the command-line help
!python -m phaseshifts.phsh --help

### Example Command-Line Usage

```bash
# Basic usage
phsh.py -b Ni111.bul -s Ni111.inp -f CLEED

# With custom energy range and output directory
phsh.py -b Ni111.bul -s Ni111.inp -f curve -r 20 500 5 -S ./output

# Using the relativistic backend for heavy elements
phsh.py -b Pt111.bul -s Pt111.inp -f CLEED --backend bvh
```

## Troubleshooting

### Common Issues

1. **"libphsh not found" error**
   - The Fortran library needs to be compiled
   - Run: `make libphsh` in the package directory

2. **Discontinuities in phase shifts**
   - Check if Step 3 (conphas) ran successfully
   - Phase shifts should be continuous functions of energy

3. **Overlapping muffin-tin spheres**
   - Reduce the muffin-tin radii in the .inp file
   - Check atomic positions for errors

4. **Memory issues**
   - Reduce l_max or energy range
   - Run on a machine with more RAM

## Summary

In this tutorial, we learned:

1. **What phase shifts are** and why they are important for LEED
2. **The Barbieri/Van Hove workflow** with its four steps
3. **How to create input files** (bulk and slab) for Ni(111)
4. **How to run calculations** using `phaseshifts.phsh.Wrapper`
5. **How to visualize** phase shift curves
6. **Key parameters** and their physical meaning

### Next Steps

- Try [Tutorial 2: ZnO Wurtzite 4-Step Workflow](02_zno_wurtzite_phshift2007.ipynb) for a more detailed walkthrough of each step
- Explore different surfaces and adsorbates
- Compare with experimental LEED data using [cleedpy](https://github.com/empa-scientific-it/cleedpy)

In [ ]:
# Cleanup (optional)
import shutil

# Uncomment to remove the temporary directory
# shutil.rmtree(work_dir)
# print(f"Cleaned up: {work_dir}")

print(f"\nWorking directory preserved at: {work_dir}")
print("Delete manually when done exploring.")